In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
model_1 = pickle.load(open('model_1.sav', 'rb'))
model_4 = pickle.load(open('model_4.sav', 'rb'))
model_5 = pickle.load(open('model_5.sav', 'rb'))

lower_bound_4 = 3.9693374713732776
upper_bound_4 = 4.858674879542743

In [3]:
test_data = pd.read_csv('test.csv')
test_data['MEAS_DT'] = pd.to_datetime(test_data['MEAS_DT'])

df_working = pd.read_csv('df_hack_final.csv')
df_working['MEAS_DT'] = pd.to_datetime(df_working['MEAS_DT'])

In [4]:
df_test = df_working[(df_working['MEAS_DT'].isin(test_data['MEAS_DT']))].dropna()

test_sample = df_test

def predict(data, diap_percent):

  def myround(x, prec=2, base=.05):
    return round(base * round(float(x) / base), prec)

  vf = np.vectorize(myround)

  fm1 = model_1.predict(data[['Ni_oreth']])

  fm1_1C_max = fm1 + fm1 * diap_percent
  fm1_1C_min = fm1 - fm1 * diap_percent

  fm1_1C_max = fm1_1C_max.round(1)
  fm1_1C_min = fm1_1C_min.round(1)

  fm1_2C_max = fm1 + fm1 * diap_percent
  fm1_2C_min = fm1 - fm1 * diap_percent

  fm1_2C_max = fm1_2C_max.round(1)
  fm1_2C_min = fm1_2C_min.round(1)

  fm4_1C_min = vf(np.repeat(np.array(lower_bound_4), len(data)))
  fm4_1C_max = vf(np.repeat(np.array(upper_bound_4), len(data)))

  fm4_2C_min = vf(np.repeat(np.array(lower_bound_4), len(data)))
  fm4_2C_max = vf(np.repeat(np.array(upper_bound_4), len(data)))

  data['Ni_4_sum'] = (data['Ni_4.1C'] + data['Ni_4.2C']) / 2
  data['Ni_4_sum_shifted'] = data['Ni_4_sum'].shift(4)

  fm5 = model_4.predict(data[['Ni_4_sum_shifted']].fillna(data[['Ni_4_sum_shifted']].mean()))

  fm5_1C_max = fm5 + fm5 * diap_percent
  fm5_1C_min = fm5 - fm5 * diap_percent

  fm5_1C_max = vf(fm5_1C_max)
  fm5_1C_min = vf(fm5_1C_min)

  fm5_2C_max = fm5 + fm5 * diap_percent
  fm5_2C_min = fm5 - fm5 * diap_percent

  fm5_2C_max = vf(fm5_2C_max)
  fm5_2C_min = vf(fm5_2C_min)

  data['Ni_5_sum'] = (data['Ni_5.1C'] + data['Ni_5.2C']) / 2
  data['Ni_5_sum_shifted'] = data['Ni_5_sum'].shift(3)

  fm6 = model_5.predict(data[['Ni_5_sum_shifted']].fillna(data[['Ni_5_sum_shifted']].mean()))

  fm6_1C_max = fm6 + fm6 * diap_percent
  fm6_1C_min = fm6 - fm6 * diap_percent

  fm6_1C_max = vf(fm6_1C_max)
  fm6_1C_min = vf(fm6_1C_min)

  fm6_2C_max = fm6 + fm6 * diap_percent
  fm6_2C_min = fm6 - fm6 * diap_percent

  fm6_2C_max = vf(fm6_2C_max)
  fm6_2C_min = vf(fm6_2C_min)

  df_result = pd.DataFrame({'MEAS_DT': data['MEAS_DT'],
                     'Ni_1.1C_min': fm1_1C_min,
                     'Ni_1.1C_max': fm1_1C_max,
                     'Ni_1.2C_max': fm1_2C_max,
                     'Ni_1.2C_min': fm1_2C_min,
                     'Ni_4.1C_min': fm4_1C_min,
                     'Ni_4.1C_max': fm4_1C_max,
                     'Ni_4.2C_min': fm4_2C_min,
                     'Ni_4.2C_max': fm4_2C_max,
                     'Ni_5.1C_max': fm5_1C_max,
                     'Ni_5.1C_min': fm5_1C_min,
                     'Ni_5.2C_max': fm5_2C_max,
                     'Ni_5.2C_min': fm5_2C_min,
                     'Ni_6.1C_min': fm6_1C_max,
                     'Ni_6.1C_max': fm6_1C_min,
                     'Ni_6.2C_max': fm6_2C_max,
                     'Ni_6.2C_min': fm6_2C_min
                     })

  return df_result

In [5]:
predict(test_sample, 0.1)

MEAS_DT  Ni_1.1C_min  Ni_1.1C_max  Ni_1.2C_max  Ni_1.2C_min  \
1777  2024-01-19 12:15:00          2.7          3.3          3.3          2.7   
1778  2024-01-19 12:30:00          2.7          3.3          3.3          2.7   
1779  2024-01-19 12:45:00          2.7          3.3          3.3          2.7   
1780  2024-01-19 13:00:00          2.7          3.3          3.3          2.7   
1781  2024-01-19 13:15:00          2.7          3.2          3.2          2.7   
...                   ...          ...          ...          ...          ...   
29732 2024-11-05 17:00:00          2.7          3.3          3.3          2.7   
29733 2024-11-05 17:15:00          2.6          3.2          3.2          2.6   
29734 2024-11-05 17:30:00          2.6          3.1          3.1          2.6   
29735 2024-11-05 17:45:00          2.5          3.1          3.1          2.5   
29738 2024-11-05 18:30:00          2.5          3.1          3.1          2.5   

       Ni_4.1C_min  Ni_4.1C_max  Ni_4.2C_min  Ni_4.2C_max  Ni_5.1C_max  \
1777          3.95         4.85         3.95         4.85          7.3   
1778          3.95         4.85         3.95         4.85          7.3   
1779          3.95         4.85         3.95         4.85          7.3   
1780          3.95         4.85         3.95         4.85          7.3   
1781          3.95         4.85         3.95         4.85          7.2   
...            ...          ...          ...          ...          ...   
29732         3.95         4.85         3.95         4.85          7.3   
29733         3.95         4.85         3.95         4.85          7.3   
29734         3.95         4.85         3.95         4.85          7.3   
29735         3.95         4.85         3.95         4.85          7.3   
29738         3.95         4.85         3.95         4.85          7.3   

       Ni_5.1C_min  Ni_5.2C_max  Ni_5.2C_min  Ni_6.1C_min  Ni_6.1C_max  \
1777          6.00          7.3         6.00        10.20         8.35   
1778          6.00          7.3         6.00        10.20         8.35   
1779          6.00          7.3         6.00        10.20         8.35   
1780          6.00          7.3         6.00        10.00         8.20   
1781          5.90          7.2         5.90         9.95         8.15   
...            ...          ...          ...          ...          ...   
29732         6.00          7.3         6.00        10.05         8.20   
29733         5.95          7.3         5.95        10.10         8.25   
29734         5.95          7.3         5.95        10.05         8.25   
29735         5.95          7.3         5.95        10.05         8.20   
29738         5.95          7.3         5.95        10.10         8.25   

       Ni_6.2C_max  Ni_6.2C_min  
1777         10.20         8.35  
1778         10.20         8.35  
1779         10.20         8.35  
1780         10.00         8.20  
1781          9.95         8.15  
...            ...          ...  
29732        10.05         8.20  
29733        10.10         8.25  
29734        10.05         8.25  
29735        10.05         8.20  
29738        10.10         8.25  

[4043 rows x 17 columns]